In [72]:
import random

# from sentence_transformers import SentenceTransformer
import pandas as pd
# import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import numpy as np
import json
import joblib
import requests

make sure server is on and the pkl files of kmeans clustes is available.

In [73]:
weights = [5,5,1,1,1,1,1,1,1]
cols = ['bedrooms', 'bathrooms', 'price', 'livingArea', 'leisure_within_5km', 'shops_within_5km', 'schools_within_5km', 'transit_within_2km', 'homeType']

Running on Sample Input

In [74]:
state = 'NY'
kmeans = joblib.load(f"{state}_kmeans.pkl")
scaler = joblib.load(f"{state}_scaler.pkl")

In [75]:
cluster_centers = kmeans.cluster_centers_

# Since we applied weights, we need to reverse this before denormalization
cluster_centers_unweighted = cluster_centers /weights

cluster_centers_to_denorm = cluster_centers_unweighted[:,:-1]
# print(cluster_centers_to_denorm)
# Denormalize the cluster centers
cluster_centers_denormalized = scaler.inverse_transform(cluster_centers_to_denorm)

cluster_centers_final= np.concatenate((cluster_centers_denormalized,cluster_centers_unweighted[:,-1].reshape(-1, 1)),axis=1)
cluster_centers_final

array([[ 1.35294118e+00,  1.29411765e+00,  1.10576471e+06,
         2.69352941e+02, -3.55271368e-15,  0.00000000e+00,
         0.00000000e+00,  5.68434189e-14,  1.00000000e+00],
       [ 3.65625000e+00,  4.21875000e+00,  6.12306250e+06,
         2.63946875e+03,  2.22187500e+01,  5.97156250e+02,
         1.68968750e+02,  2.78968750e+02,  1.28125000e+00],
       [ 2.01923077e+00,  1.74038462e+00,  1.71812018e+06,
         7.76278750e+02,  2.42211538e+01,  5.79605769e+02,
         1.73567308e+02,  2.80000000e+02,  9.61538462e-01],
       [ 1.00000000e+00,  1.00000000e+00,  9.32013599e+05,
         3.69088300e+02,  2.33265306e+01,  6.19387755e+02,
         1.72707483e+02,  2.87965986e+02,  9.65986395e-01],
       [ 2.00000000e+00,  1.20000000e+01,  3.98000000e+06,
         1.28300000e+03,  2.10000000e+01,  6.38000000e+02,
         1.65000000e+02,  3.48000000e+02,  1.00000000e+00],
       [ 9.00000000e+00,  5.50000000e+00,  1.99950000e+06,
         3.86000000e+03,  4.40000000e+01,  2.450000

In [76]:
df_cluster_centers=pd.DataFrame(cluster_centers_final,columns=list(cols))
np.floor(df_cluster_centers)

,bedrooms,bathrooms,price,livingArea,leisure_within_5km,shops_within_5km,schools_within_5km,transit_within_2km,homeType
0,1.0,1.0,1105764.0,269.0,-1.0,0.0,0.0,0.0,1.0
1,3.0,4.0,6123062.0,2639.0,22.0,597.0,168.0,278.0,1.0
2,2.0,1.0,1718120.0,776.0,24.0,579.0,173.0,280.0,0.0
3,1.0,1.0,932013.0,369.0,23.0,619.0,172.0,287.0,0.0
4,2.0,12.0,3980000.0,1283.0,21.0,638.0,165.0,348.0,1.0
5,9.0,5.0,1999500.0,3860.0,44.0,245.0,194.0,254.0,3.0
6,0.0,1.0,605358.0,259.0,22.0,648.0,169.0,294.0,0.0
7,4.0,5.0,10024705.0,4059.0,20.0,598.0,170.0,273.0,1.0
8,6.0,7.0,16227857.0,5211.0,23.0,565.0,168.0,279.0,2.0
9,2.0,3.0,3337566.0,1402.0,23.0,639.0,173.0,319.0,0.0


In [77]:
predict_X=[2,1,1700000,2000,20,400,250,200,2]  # give input here
df_predict = pd.DataFrame([predict_X],columns=cols)
scaler_predict=StandardScaler()

df_to_transform_p = df_predict.drop(columns=['homeType'])
df_excluded_p = df_predict[['homeType']]
# Apply the transformation
transformed_array_p = scaler.transform(df_to_transform_p)
print(transformed_array_p)
df_transformed_p = pd.DataFrame(transformed_array_p, columns=df_to_transform_p.columns, index=df_predict.index)
# Recombine the DataFrame
df_result_p = pd.concat([df_transformed_p, df_excluded_p], axis=1)
# print(df_result)
X_p=df_result_p.values
X_p=X_p*weights
X_p

[[ 0.09090596 -0.73108393 -0.20746602  0.8198349  -0.35234394 -0.98294832
   2.22322311 -0.88709456]]


array([[ 0.45452982, -3.65541963, -0.20746602,  0.8198349 , -0.35234394,
        -0.98294832,  2.22322311, -0.88709456,  2.        ]])

In [78]:
kmeans.predict(X_p)

array([2])

Running on Server Response

In [79]:
homeType_Encoding= {
        'APARTMENT': 0,
        'CONDO': 1,
        'MANUFACTURED': 2,
        'MULTI_FAMILY': 3,
        'SINGLE_FAMILY': 4,
        'TOWNHOUSE': 5
    }

In [80]:
state = 'NY'
kmeans = joblib.load(f"{state}_kmeans.pkl")
scaler = joblib.load(f"{state}_scaler.pkl")
weights = [5,5,1,1,1,1,1,1,1]


In [81]:
json_file_path = "./ClusterPoints_Dataset.json"

with open(json_file_path, "r") as file:
    kmeansdata = json.load(file)

json_file_path_1 = "./Labelled_Dataset_with_zpid.json"

with open(json_file_path_1, "r") as file:
    fulldata = json.load(file)

In [82]:
predict_X=[2,1,1700000,2000,20,400,250,200,2]  # give input here
df_predict = pd.DataFrame([predict_X],columns=cols)
scaler_predict=StandardScaler()

df_to_transform_p = df_predict.drop(columns=['homeType'])
df_excluded_p = df_predict[['homeType']]
# Apply the transformation
transformed_array_p = scaler.transform(df_to_transform_p)
print(transformed_array_p)
df_transformed_p = pd.DataFrame(transformed_array_p, columns=df_to_transform_p.columns, index=df_predict.index)
# Recombine the DataFrame
df_result_p = pd.concat([df_transformed_p, df_excluded_p], axis=1)
# print(df_result)
X_p=df_result_p.values
X_p=X_p*weights
X_p

[[ 0.09090596 -0.73108393 -0.20746602  0.8198349  -0.35234394 -0.98294832
   2.22322311 -0.88709456]]


array([[ 0.45452982, -3.65541963, -0.20746602,  0.8198349 , -0.35234394,
        -0.98294832,  2.22322311, -0.88709456,  2.        ]])

In [83]:
predicted_label=kmeans.predict(X_p)
predicted_label

array([2])

In [84]:
state_full_data=fulldata[state]
state_full_data

[{'zpid': 31491229,
  'bedrooms': 1,
  'bathrooms': 1,
  'price': 650000.0,
  'homeType': 1,
  'livingArea': 776.0,
  'leisure_within_5km': 21,
  'shops_within_5km': 540,
  'schools_within_5km': 128,
  'transit_within_2km': 121,
  'labels': 3},
 {'zpid': 31492996,
  'bedrooms': 2,
  'bathrooms': 2,
  'price': 959000.0,
  'homeType': 1,
  'livingArea': 1000.0,
  'leisure_within_5km': 19,
  'shops_within_5km': 538,
  'schools_within_5km': 127,
  'transit_within_2km': 110,
  'labels': 2},
 {'zpid': 31493494,
  'bedrooms': 1,
  'bathrooms': 1,
  'price': 1100000.0,
  'homeType': 1,
  'livingArea': 815.0,
  'leisure_within_5km': 20,
  'shops_within_5km': 614,
  'schools_within_5km': 141,
  'transit_within_2km': 194,
  'labels': 3},
 {'zpid': 31494186,
  'bedrooms': 0,
  'bathrooms': 1,
  'price': 750000.0,
  'homeType': 1,
  'livingArea': 465.0,
  'leisure_within_5km': 22,
  'shops_within_5km': 717,
  'schools_within_5km': 160,
  'transit_within_2km': 272,
  'labels': 6},
 {'zpid': 31494718

In [85]:
def filter_by_labels(data, labels):
    # Use list comprehension to get dictionaries with matching labels
    filtered_data = [entry for entry in data if entry['labels'] == labels]
    return filtered_data

In [86]:
state_filtered_data = filter_by_labels(state_full_data, predicted_label[0])
len(state_filtered_data)

104

In [87]:
state_filtered_data

[{'zpid': 31492996,
  'bedrooms': 2,
  'bathrooms': 2,
  'price': 959000.0,
  'homeType': 1,
  'livingArea': 1000.0,
  'leisure_within_5km': 19,
  'shops_within_5km': 538,
  'schools_within_5km': 127,
  'transit_within_2km': 110,
  'labels': 2},
 {'zpid': 31495264,
  'bedrooms': 2,
  'bathrooms': 1,
  'price': 798000.0,
  'homeType': 1,
  'livingArea': 525.0,
  'leisure_within_5km': 28,
  'shops_within_5km': 836,
  'schools_within_5km': 181,
  'transit_within_2km': 268,
  'labels': 2},
 {'zpid': 31499051,
  'bedrooms': 2,
  'bathrooms': 2,
  'price': 3500000.0,
  'homeType': 1,
  'livingArea': 1408.0,
  'leisure_within_5km': 23,
  'shops_within_5km': 632,
  'schools_within_5km': 166,
  'transit_within_2km': 320,
  'labels': 2},
 {'zpid': 31507674,
  'bedrooms': 2,
  'bathrooms': 1,
  'price': 1350000.0,
  'homeType': 1,
  'livingArea': 1053.0,
  'leisure_within_5km': 21,
  'shops_within_5km': 762,
  'schools_within_5km': 182,
  'transit_within_2km': 346,
  'labels': 2},
 {'zpid': 31511

Sorting Entries based on distance from cluster

In [89]:
df = pd.DataFrame(state_filtered_data)
df=df.set_index(keys=['zpid'])
df

,bedrooms,bathrooms,price,homeType,livingArea,leisure_within_5km,shops_within_5km,schools_within_5km,transit_within_2km,labels
zpid,,,,,,,,,,
31492996,2,2,959000.0,1,1000.000000,19,538,127,110,2
31495264,2,1,798000.0,1,525.000000,28,836,181,268,2
31499051,2,2,3500000.0,1,1408.000000,23,632,166,320,2
31507674,2,1,1350000.0,1,1053.000000,21,762,182,346,2
31511219,2,2,1100000.0,1,963.000000,22,789,197,319,2
...,...,...,...,...,...,...,...,...,...,...
2078380933,2,2,2688000.0,1,1170.000000,29,888,182,244,2
2089719603,2,2,1695000.0,1,1036.000000,21,569,158,242,2
2112868113,2,1,850000.0,1,0.000000,21,744,182,352,2


In [90]:
df_data = df.drop(columns=['labels'])
df_home= df_data['homeType']
df_data=df_data.drop(columns=['homeType'])
df_final = pd.concat((df_data, df_home),axis=1)
df_final

,bedrooms,bathrooms,price,livingArea,leisure_within_5km,shops_within_5km,schools_within_5km,transit_within_2km,homeType
zpid,,,,,,,,,
31492996,2,2,959000.0,1000.000000,19,538,127,110,1
31495264,2,1,798000.0,525.000000,28,836,181,268,1
31499051,2,2,3500000.0,1408.000000,23,632,166,320,1
31507674,2,1,1350000.0,1053.000000,21,762,182,346,1
31511219,2,2,1100000.0,963.000000,22,789,197,319,1
...,...,...,...,...,...,...,...,...,...
2078380933,2,2,2688000.0,1170.000000,29,888,182,244,1
2089719603,2,2,1695000.0,1036.000000,21,569,158,242,1
2112868113,2,1,850000.0,0.000000,21,744,182,352,1


In [91]:
np.unique(df_final['bathrooms'])

array([1, 2], dtype=int64)

In [92]:
df_to_transform = df_final.drop(columns=['homeType'])
df_excluded= df_final[['homeType']]
# Apply the transformation
transformed_array = scaler.transform(df_to_transform)
df_transformed = pd.DataFrame(transformed_array, columns=df_to_transform.columns, index=df_final.index)
# Recombine the DataFrame
df_result= pd.concat([df_transformed, df_excluded], axis=1)
# print(df_result)
X=df_result.values
X=X*weights
X[:,1]

array([-0.21340491, -3.65541963, -0.21340491, -3.65541963, -0.21340491,
       -0.21340491, -0.21340491, -3.65541963, -0.21340491, -0.21340491,
       -0.21340491, -3.65541963, -3.65541963, -0.21340491, -3.65541963,
       -0.21340491, -0.21340491, -0.21340491, -0.21340491, -3.65541963,
       -3.65541963, -3.65541963, -0.21340491, -0.21340491, -0.21340491,
       -0.21340491, -0.21340491, -0.21340491, -0.21340491, -0.21340491,
       -0.21340491, -0.21340491, -3.65541963, -0.21340491, -0.21340491,
       -0.21340491, -0.21340491, -3.65541963, -0.21340491, -0.21340491,
       -0.21340491, -0.21340491, -3.65541963, -0.21340491, -0.21340491,
       -0.21340491, -0.21340491, -3.65541963, -0.21340491, -3.65541963,
       -0.21340491, -0.21340491, -0.21340491, -3.65541963, -0.21340491,
       -0.21340491, -0.21340491, -0.21340491, -0.21340491, -0.21340491,
       -0.21340491, -0.21340491, -0.21340491, -3.65541963, -3.65541963,
       -0.21340491, -0.21340491, -0.21340491, -0.21340491, -3.65

In [93]:
cluster_centers = kmeans.cluster_centers_
cluster_of_interest = cluster_centers[predicted_label[0]]
cluster_of_interest

array([ 0.52502142, -1.10700489, -0.20199347, -0.1428045 ,  0.26581815,
       -0.04790749,  0.18704836,  0.0099346 ,  0.96153846])

In [94]:
X = np.array(X)

Y = np.array(cluster_of_interest)

# Calculate Euclidean distances
distances = np.linalg.norm(X - Y, axis=1)

# Get the indices that would sort the distances
sorted_indices = np.argsort(distances)
row_indexes = df_final.index[sorted_indices]
# Sort the 2D matrix based on the distances
sorted_X = X[sorted_indices]


In [95]:
print("\nSorted matrix:")
print(sorted_X[:,1])


Sorted matrix:
[-0.21340491 -0.21340491 -0.21340491 -0.21340491 -0.21340491 -0.21340491
 -0.21340491 -0.21340491 -0.21340491 -0.21340491 -0.21340491 -0.21340491
 -0.21340491 -0.21340491 -0.21340491 -0.21340491 -0.21340491 -0.21340491
 -0.21340491 -0.21340491 -0.21340491 -0.21340491 -0.21340491 -0.21340491
 -0.21340491 -0.21340491 -0.21340491 -0.21340491 -0.21340491 -0.21340491
 -0.21340491 -0.21340491 -0.21340491 -0.21340491 -0.21340491 -0.21340491
 -0.21340491 -0.21340491 -0.21340491 -0.21340491 -0.21340491 -0.21340491
 -0.21340491 -0.21340491 -0.21340491 -0.21340491 -0.21340491 -0.21340491
 -0.21340491 -0.21340491 -0.21340491 -0.21340491 -0.21340491 -0.21340491
 -0.21340491 -0.21340491 -0.21340491 -0.21340491 -0.21340491 -0.21340491
 -0.21340491 -0.21340491 -0.21340491 -0.21340491 -0.21340491 -0.21340491
 -0.21340491 -0.21340491 -0.21340491 -0.21340491 -0.21340491 -3.65541963
 -3.65541963 -3.65541963 -0.21340491 -3.65541963 -3.65541963 -0.21340491
 -3.65541963 -3.65541963 -3.6554196

In [96]:
sortedX = sorted_X /weights

X_to_denorm = sortedX[:,:-1]
# print(cluster_centers_to_denorm)
# Denormalize the cluster centers
X_denormalized = scaler.inverse_transform(X_to_denorm)

X_final= np.concatenate((X_denormalized,sortedX[:,-1].reshape(-1, 1)),axis=1)
X_final

array([[2.00000000e+00, 2.00000000e+00, 1.47500000e+06, 9.25000000e+02,
        2.00000000e+01, 5.63000000e+02, 1.66000000e+02, 2.47000000e+02,
        1.00000000e+00],
       [2.00000000e+00, 2.00000000e+00, 1.69500000e+06, 1.03600000e+03,
        2.10000000e+01, 5.69000000e+02, 1.58000000e+02, 2.42000000e+02,
        1.00000000e+00],
       [2.00000000e+00, 2.00000000e+00, 1.34500000e+06, 9.58000000e+02,
        2.10000000e+01, 6.16000000e+02, 1.56000000e+02, 3.12000000e+02,
        1.00000000e+00],
       [2.00000000e+00, 2.00000000e+00, 1.99500000e+06, 1.13686838e-13,
        2.10000000e+01, 5.86000000e+02, 1.76000000e+02, 3.00000000e+02,
        1.00000000e+00],
       [2.00000000e+00, 2.00000000e+00, 1.65000000e+06, 1.25400000e+03,
        2.10000000e+01, 6.16000000e+02, 1.56000000e+02, 3.12000000e+02,
        1.00000000e+00],
       [2.00000000e+00, 2.00000000e+00, 2.06000000e+06, 1.38300000e+03,
        2.10000000e+01, 5.89000000e+02, 1.51000000e+02, 2.79000000e+02,
        1.0

In [97]:
df_X=pd.DataFrame(X_final,columns=list(cols))
df_sorted=np.floor(df_X)
df_sorted=df_sorted.set_index(row_indexes)
df_sorted

,bedrooms,bathrooms,price,livingArea,leisure_within_5km,shops_within_5km,schools_within_5km,transit_within_2km,homeType
zpid,,,,,,,,,
31521020,2.0,2.0,1475000.0,925.0,20.0,563.0,166.0,247.0,1.0
2089719603,2.0,2.0,1695000.0,1036.0,21.0,569.0,158.0,242.0,1.0
244701164,2.0,2.0,1345000.0,958.0,21.0,616.0,156.0,312.0,1.0
79498426,2.0,2.0,1995000.0,0.0,21.0,586.0,176.0,300.0,1.0
244746432,2.0,2.0,1650000.0,1254.0,21.0,616.0,156.0,312.0,1.0
...,...,...,...,...,...,...,...,...,...
72534234,3.0,1.0,885000.0,1212.0,28.0,402.0,205.0,345.0,1.0
244793565,2.0,1.0,825000.0,0.0,41.0,111.0,156.0,99.0,1.0
119911138,2.0,1.0,529000.0,1000.0,37.0,80.0,115.0,103.0,1.0
